In [16]:
import requests
from dotenv import load_dotenv
import os
import faiss
from sentence_transformers import SentenceTransformer
import torch
from datasets import load_dataset
from sentence_transformers import CrossEncoder
from transformers import BertTokenizer, BertForSequenceClassification
from typing import Literal
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import plotly.express as px
import pandas as pd

load_dotenv()

API_KEY = os.getenv("CLARIN_API_KEY")

MODELS_ENDPOINT = "https://services.clarin-pl.eu/api/v1/oapi/models"
COMPLETIONS_ENDPOINT = "https://services.clarin-pl.eu/api/v1/oapi/chat/completions"

def get_models():
    headers = {
        'Authorization': f'Bearer {API_KEY}',  
        'Content-Type': 'application/json'
    }
    response = requests.get(MODELS_ENDPOINT, headers=headers)  
    if response.status_code == 200:
        return response.json()  

models = get_models()
print(models)


{'data': [{'id': 'bielik', 'full_name': 'speakleash/Bielik-11B-v2.2-Instruct', 'name': 'speakleash/Bielik-11B-v2.2-Instruct'}, {'id': 'cohere', 'full_name': 'CohereForAI/c4ai-command-r-plus', 'name': 'CohereForAI/c4ai-command-r-plus'}, {'id': 'llama3.1-8b', 'full_name': 'meta-llama/Llama-3.1-8B-Instruct', 'name': 'meta-llama/Llama-3.1-8B-Instruct'}, {'id': 'llama', 'full_name': 'meta-llama/Llama-3.1-8B-Instruct', 'name': 'meta-llama/Llama-3.1-8B-Instruct'}, {'id': 'llama-guard', 'full_name': 'meta-llama/Llama-Guard-3-8B', 'name': 'meta-llama/Llama-Guard-3-8B'}, {'id': 'llama3.1', 'full_name': 'meta-llama/Meta-Llama-3.1-70B-Instruct', 'name': 'meta-llama/Meta-Llama-3.1-70B-Instruct'}, {'id': 'openchat', 'full_name': 'openchat/openchat-3.5-1210', 'name': 'openchat/openchat-3.5-1210'}, {'id': 'mixtral-8x22B', 'full_name': 'mistralai/Mixtral-8x22B-Instruct-v0.1', 'name': 'mistralai/Mixtral-8x22B-Instruct-v0.1'}]}


In [17]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_name = 'cross-encoder/ms-marco-MiniLM-L-6-v2'

In [18]:
class Reranker():
    
    def __init__(self, model_name: str, max_length: int = 512) -> None:
        self.model_name = model_name
        self.max_length = max_length
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        
        self.model_rank = CrossEncoder(self.model_name, 
                                       max_length = self.max_length, 
                                       device = self.device)
        
        self.tokenizer = BertTokenizer.from_pretrained(self.model_name, 
                                                       max_length = self.max_length)
    
    
    def rank_answers(self, query: str, answers: list[str], top_k: int = 5) -> list[dict[Literal['corpus_id', 'score', 'text'], int | float | str]]:
        self.query = query
        self.top_k = top_k
        self.answers_ranking = self.model_rank.rank(query, 
                                                    answers,
                                                    top_k = 300,
                                                    return_documents = True,
                                                    show_progress_bar = True)
        
        print(f'Zapytanie: {self.query}')
        for k in range(self.top_k):
            print(f'Odpowiedź {k + 1}: {self.answers_ranking[k]["text"]}')
    
    
    def find_embeddings(self) -> list[dict[torch.Tensor, str]]:
        question = self.query
        answers = [dct['text'] for dct in self.answers_ranking]
        texts = [question, *answers]
        labels = ['Zapytanie', *[f'Top {self.top_k} odpowiedzi'] * self.top_k, *['Pozostałe odpowiedzi'] * (len(answers) - self.top_k)]
        tensors = []
        
        for text in texts:
            inputs = self.tokenizer(text, 
                                return_tensors = 'pt', 
                                truncation = True, 
                                padding = True, 
                                max_length = self.max_length).to(device)
            
            with torch.no_grad():
                outputs = self.model_rank.model.bert(**inputs)
                
            embedding = outputs.last_hidden_state[:, 0, :].squeeze()   # EMBEDDING TOKENU [CLS]
            # embedding = outputs.last_hidden_state.mean(dim = 1).squeeze()   # ŚREDNIA Z EMBEDDINGÓW DLA WSZYSTKICH TOKENÓW
            tensors.append(embedding)
        
        emb_tensor = torch.stack(tensors)
        
        self.emb_tensor = emb_tensor
        self.texts = texts
        self.labels = labels
    
    
    def plot_embeddings(self) -> None:
            
        embs, texts, labels  = self.emb_tensor, self.texts, self.labels
        pca = TSNE(n_components = 2, perplexity = 10)
        pca_embs = pca.fit_transform(embs.cpu().numpy())
        
        df = pd.DataFrame({'x': pca_embs[:, 0], 'y': pca_embs[:, 1], 'texts': texts, 'label': labels})
        df['texts'] = df['texts'].apply(lambda text: text[:100] + '...' if len(text) > 100 else text)
        
        
        fig = px.scatter(df, x = 'x', y = 'y', color = 'label',
            title = 'Wizualizacja osadzeń dla modelu re-ranker',
            labels = {'label': 'Legenda'},
            hover_name = df['texts']
        )
        fig.update_traces(marker = dict(size = 10), selector = dict(mode = 'markers'))
        fig.show()

In [6]:
from datasets import load_dataset

dataset = load_dataset("stanfordnlp/imdb")

Generating unsupervised split: 100%|██████████| 50000/50000 [00:00<00:00, 204082.32 examples/s]


In [7]:
def build_faiss_index(dataset, model):
    reviews = dataset["train"]["text"] 
    embeddings = model.encode(reviews, convert_to_numpy=True).astype("float32")

    index = faiss.IndexFlatL2(embeddings.shape[1])  
    index.add(embeddings)  
    return index, reviews


In [ ]:
def search_faiss(query, k=5):
    query_embedding = model.encode([query], convert_to_numpy=True).astype('float32')
    distances, indices = index.search(query_embedding, k)

    result_reviews = [reviews[idx] for idx in indices[0]]
    return distances[0], result_reviews


In [ ]:
def clarin_chat_completion(model_id, prompt, max_tokens=200):
    url = f"{CLARIN_BASE_URL}/chat/completions"
    headers = {"Authorization": f"Bearer {API_KEY}", "Content-Type": "application/json"}
    data = {
        "model": model_id,
        "messages": [{"role": "user", "content": prompt}],
        "max_tokens": max_tokens
    }
    response = requests.post(url, json=data, headers=headers)
    if response.status_code == 200:
        return response.json()["choices"][0]["message"]["content"]
    else:
        raise Exception(f"CLARIN API Error: {response.status_code} - {response.text}")

In [23]:
def handle_query_with_rag(user_query, model_id="bielik", k=5):
    distances, result_reviews = search_faiss(user_query, k)

    context = "\n\n".join([f"Recenzja {i+1}: {review}" for i, review in enumerate(result_reviews)])
    if not context.strip():
        context = "Brak dodatkowego kontekstu."

    prompt = f"Zapytanie: {user_query}\n\nKontekst:\n{context}\n\nOdpowiedz w kontekście powyższych informacji."
    response = clarin_chat_completion(model_id, prompt)
    return response


In [13]:
model = SentenceTransformer("all-MiniLM-L6-v2")
index, reviews = build_faiss_index(dataset, model)

In [15]:
faiss.write_index(index, 'index/imdb_faiss.bin')

In [19]:
user_query = "What is the best movie to watch with friends?"
response = handle_query_with_rag(user_query)
print("Odpowiedź:", response)

Odpowiedź: Wybór najlepszego filmu do oglądania z przyjaciółmi zależy od kilku czynników, takich jak gust grupy, okazja oraz tematyka filmu. Ze swojej perspektywy mogę polecić kilka tytułów, które nadają się znakomicie na seans w towarzystwie znajomych.

Jeśli szukasz lekkiego i pełnego humoru filmu, świetną opcją będą komedie romantyczne, takie jak "Notting Hill" czy "Dziewczyna z sąsiedztwa". Oba te filmy nie tylko zapewnią sporo śmiechu, ale także stworzą miłą atmosferę do rozmowy po projekcji.

Dla fanów animacji polecam "Toy Story", serię filmów o Toy Story jest niezwykle uroczym i uniwersalnym wyborem, który spodoba się widzom w każdym wieku. Filmy te mają znakomitą fabułę oraz piękne przesłanie, a ich różnorodność sprawi, że nikt się nie znudzi.

Miłośnicy kina akcji mogą cieszyć się przygodami w cyklu "Mad Max" - te niezwykle dynamiczne i pełne spektakularnych efektów specjalnych produkcje zapewnią mocne wrażenia i dodadzą energii do spotkania ze znajomymi.

A jeśli chcesz wpro

In [24]:
handle_query_with_rag("What do people think about horror movies?")

'W oparciu o podane informacje, nie możemy bezpośrednio określić dokładnej opinii ludzi o horrorach, ponieważ zamiast treści recenzji mamy tylko numery referencyjne. Jednakże na ich podstawie możemy sformułować następujące domysły:\n\n1. Skoro są to recenzje filmów grozy, prawdopodobnie ludzie mają różnorodne opinie - od negatywnych po entuzjastyczne. \n\n2. Numery recenzji sugerują, że opinie byłyby rozłożone na kilka tysięcy widzów (np. dla recenzji 23708 - prawie 24 tysiące). W tak dużej grupie spodziewalibyśmy się również pogłębionej różnorodności preferencji i emocji wobec horrorów.\n\n3. Możliwe, że niektóre z tych recenzji dotyczą klasyków gatunku jak "Psychoza" czy "Piła", inne mogłyby omawiać najnowsze produkcje lub niszowe kino grozy.\n\nAby uzyskać szczegółową opinię publiczną na temat horrorów, potrzebne byłyby pełne treści tych recenzji oraz dodatkowe dane ze sprzedaży biletów, liczby wyświetleń czy reakcji w mediach społecznościowych. Jednocześnie pamiętajmy, że opinie os

______________

In [ ]:
import requests
from dotenv import load_dotenv
import os
import faiss
import torch
from sentence_transformers import SentenceTransformer, CrossEncoder
from transformers import pipeline
from datasets import load_dataset

load_dotenv()
API_KEY = os.getenv("CLARIN_API_KEY")


CLARIN_BASE_URL = "https://services.clarin-pl.eu/api/v1/oapi"


def get_models():
    headers = {'Authorization': f'Bearer {API_KEY}', 'Content-Type': 'application/json'}
    response = requests.get(f"{CLARIN_BASE_URL}/models", headers=headers)
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception(f"Failed to fetch models: {response.status_code} - {response.text}")

dataset = load_dataset("stanfordnlp/imdb")

def build_faiss_index(dataset, model):
    reviews = dataset["train"]["text"]
    embeddings = model.encode(reviews, convert_to_numpy=True).astype("float32")
    
    d = embeddings.shape[1]  
    quantizer = faiss.IndexFlatL2(d)  

    nlist = 100 
    m = 8  
    nbits = 8  
    index = faiss.IndexIVFPQ(quantizer, d, nlist, m, nbits)  

    index.train(embeddings)  
    index.add(embeddings)  

    faiss.write_index(index, "index/imdb_faiss_ivfpq.bin")
    print("Indeks został zapisany na dysk.")

    return index, reviews


def search_faiss(index, query, model, reviews, k=5):
    query_embedding = model.encode([query], convert_to_numpy=True).astype("float32")
    distances, indices = index.search(query_embedding, k)
    result_reviews = [reviews[idx] for idx in indices[0]]
    return distances[0], result_reviews

def rerank_with_cross_encoder(query, result_reviews, reranker, k=5):

    pairs = [[query, review] for review in result_reviews]

    rerank_scores = reranker.predict(pairs)

    ranked_reviews = [result_reviews[i] for i in sorted(range(len(rerank_scores)), key=lambda i: rerank_scores[i], reverse=True)]
    return ranked_reviews[:k]

def clarin_chat_completion(model_id, prompt, max_tokens=300):
    url = f"{CLARIN_BASE_URL}/chat/completions"
    headers = {"Authorization": f"Bearer {API_KEY}", "Content-Type": "application/json"}
    data = {
        "model": model_id,
        "messages": [{"role": "user", "content": prompt}],
        "max_tokens": max_tokens
    }
    response = requests.post(url, json=data, headers=headers)
    if response.status_code == 200:
        return response.json()["choices"][0]["message"]["content"]
    else:
        raise Exception(f"CLARIN API Error: {response.status_code} - {response.text}")

model = SentenceTransformer("all-MiniLM-L6-v2")

index, reviews = build_faiss_index(dataset, model)


Indeks został zapisany na dysk.


In [ ]:
def handle_query_with_rag(user_query,reranker, model_id="mixtral-8x22B", k=8):
    distances, result_reviews = search_faiss(index, user_query, model, reviews, 50)

    reranked_reviews = rerank_with_cross_encoder(user_query, result_reviews, reranker, k)

    context = "\n\n".join([f"Recenzja {i+1}: {review}" for i, review in enumerate(reranked_reviews)])
    if not context.strip():
        context = "Brak dodatkowego kontekstu."

    prompt = f"Zapytanie: {user_query}\n\nKontekst:\n{context}\n\nOdpowiedz zwięźle w kontekście powyższych informacji."
    response = clarin_chat_completion(model_id, prompt)
    return response, context

def evaluate_response_with_llama(question, context, answer, model_id="bielik"):

    prompt = f"Na podstawie poniższego kontekstu i pytania, oceń, jak trafna i spójna jest odpowiedź modelu '{answer}' w odniesieniu do pytania '{question}':\n\nKontekst: {context}\n\nCzy odpowiedź jest trafna i zgodna z kontekstem? Oceń odpowiedź na skali od 1 do 10, gdzie 1 to 'bardzo nietrafna', a 10 to 'bardzo trafna'. Dodaj krótki komentarz co poprawić."

    evaluation = clarin_chat_completion(model_id, prompt)
    return evaluation

In [ ]:
reranker = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2")

user_query = "What are the worst movies?"

response, context = handle_query_with_rag(user_query, reranker)

print("Użyty kontekst:\n", context)

evaluation = evaluate_response_with_llama(user_query, context, response)

print("Odpowiedź:", response)
print("Ocena:", evaluation)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Użyty kontekst:
 Recenzja 1: This is hands down the worst movie of all time. A combination of Whoopie Goldberg (the worst actress/person in history) and a talking dinosaur ala Jar-Jar-Binks add up to a painfully bad movie. That was an understatement. This movie is unwatchable. For the love of God, do not watch this movie.

Recenzja 2: The thing about calling "House of the Dead" the worst movie of all time is that it's really not. There are worse movies out there. I watch alot of Hong Kong ninja movies that are basically the result of an unfinished Japanese police drama having footage of ninjas inserted at the end to create something that could technically be called "a movie."<br /><br />House of the Dead is however one of the worst films I've ever seen at the theatres. Walking out half way through, I actually felt I was somewhat dumber for having set through 45 minutes of this piece of garbage.

Recenzja 3: This is surely one of the worst films ever made. Each scene is painful. You wil